In [1]:
!pip install xlrd==2.0.1
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/APCLab/jieba-tw.git to /tmp/pip-req-build-fh6l1kla
  Running command git clone --filter=blob:none --quiet https://github.com/APCLab/jieba-tw.git /tmp/pip-req-build-fh6l1kla
  Resolved https://github.com/APCLab/jieba-tw.git to commit 2bd4c0913cdadc865879b21a00c78e28ef1ba2c5
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.39.1-py3-none-any.whl size=6603859 sha256=02295bc742c2b00166605d2de2dd0d1171

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import xlrd
import pandas as pd
import tensorflow
import numpy as np

In [4]:
# 前處理
import csv
from tqdm import tqdm, trange
sentence_list = []
bert_label = []
with open('/content/drive/MyDrive/ToxicComment.csv', encoding="ISO-8859-1") as csvfile:
    reader = csv.reader(csvfile)
    for row in tqdm(reader):
        bert_label.append(row[1])
        sentence_list.append(row[0].lower().strip(" !@#$%^&*()_+=/'\-0987654321[]{};:"))

31000it [00:00, 31177.26it/s]


In [5]:
# 下載pretrained model
PRETRAINED_MODEL = 'bert-base-uncased'

# load pretrained bert model
from transformers import BertTokenizerFast, TFBertForSequenceClassification # 記得先pip install transformers
tokenizer = BertTokenizerFast.from_pretrained(PRETRAINED_MODEL) #設定tokenizer分詞器，這句跑一次就好
model = TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=1) #設定pretrained model，這句一樣跑一次就好

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### from_pretrained() API
[Link: from_pretrained()](https://huggingface.co/docs/transformers/v4.25.1/en/main_classes/model#transformers.PreTrainedModel.from_pretrained)

Since bert_label is a 1D list, we set `num_labels = 1`

In [6]:
# 把pretrain model拿來fine_tuning，輸入我們的惡毒評論文字，變成我們要用的惡毒評論model (AKA遷移式學習)
def bert_encoder(blackmails):
    encoded = tokenizer(blackmails, truncation=True, max_length=200, pad_to_max_length=True)
    return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']
bert_train = []
for i in tqdm(sentence_list):
    bert_train.append(bert_encoder(i))

  0%|          | 0/31000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 31000/31000 [00:14<00:00, 2141.97it/s]


In [7]:
bert_train = np.array(bert_train)
bert_label = np.array(bert_label)

print(bert_train.shape, bert_label.shape) 

(31000, 3, 200) (31000,)


In [8]:
# 91分配
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(bert_train, bert_label, test_size = 0.1, random_state = 927)
print(x_train.shape, y_train.shape)


y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)


(27900, 3, 200) (27900,)


In [9]:
# import random

# # Separate the majority and minority classes
# majority_samples = bert_train[bert_label == 0]
# minority_samples = bert_train[bert_label == 1]

# # Determine the number of samples in the minority class
# num_minority_samples = len(minority_samples)

# # Undersample the majority class
# undersampled_majority = random.sample(list(majority_samples), num_minority_samples)

# # Combine the undersampled majority class with the minority class
# undersampled_data = np.concatenate((undersampled_majority, minority_samples))
# undersampled_labels = np.concatenate((np.zeros(num_minority_samples), np.ones(num_minority_samples))).astype(int)

# print("Undersampled Data Shape:", undersampled_data.shape)
# print("Undersampled Labels Shape:", undersampled_labels.shape)


In [10]:
# from sklearn.model_selection import train_test_split

# x_train, x_val, y_train, y_val = train_test_split(undersampled_data, undersampled_labels, test_size=0.1, random_state=927)
# print(x_train.shape, y_train.shape)

# y_train = np.array(y_train, dtype=np.float32)
# y_val = np.array(y_val, dtype=np.float32)


In [11]:
# 優化數據(確保不會有bug)
tr_reviews, tr_segments, tr_masks = np.split(x_train, 3, axis=1)
val_reviews, val_segments, val_masks = np.split(x_val, 3, axis=1)
tr_reviews = tr_reviews.squeeze()
tr_segments = tr_segments.squeeze()
tr_masks = tr_masks.squeeze()
val_reviews = val_reviews.squeeze()
val_segments = val_segments.squeeze()
val_masks = val_masks.squeeze()
def example_to_features(input_ids, attention_masks, token_type_ids, y):
    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "token_type_ids": token_type_ids
    }, y


train_ds = tensorflow.data.Dataset.from_tensor_slices((tr_reviews, tr_masks, tr_segments, y_train)).map(example_to_features).shuffle(100).batch(16)
valid_ds = tensorflow.data.Dataset.from_tensor_slices((val_reviews, val_masks, val_segments, y_val)).map(example_to_features).shuffle(100).batch(16)

print(train_ds)
print(valid_ds)

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 200), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 200), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 200), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 200), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 200), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 200), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


In [12]:
tr_masks

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [13]:
# 指定訓練參數並進行訓練：
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=1e-5)
loss = tensorflow.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
bert_history = model.fit(train_ds, epochs=2, validation_data=valid_ds)

Epoch 1/2
1744/1744 [==============================] - 1285s 707ms/step - loss: 0.1935 - accuracy: 0.9177 - val_loss: 0.1620 - val_accuracy: 0.9381
Epoch 2/2
1744/1744 [==============================] - 1219s 699ms/step - loss: 0.1103 - accuracy: 0.9578 - val_loss: 0.1997 - val_accuracy: 0.9361


In [14]:
# import requests 
# import bs4

# def getContent(URL):
#     response = requests.get(URL)
#     # 將抓下來的原始碼用BeautifulSoup解析
#     soup = bs4.BeautifulSoup(response.text,"html.parser")
#     # 找到maintainer的id
#     main_container = soup.find(id='main-container')
#     all_text = main_container.text
#     # 只擷取內文
#     pre_text = all_text.split('--')[0]
#     texts = pre_text.split('\n')
#     contents = texts[2:]
#     content = '\n'.join(contents)
#     return content
# 輸入自訂惡毒評論語句，並且進行前處理。
# URL = input("請輸入一ptt網頁欲分析情勒比重的網址: ")
# blackmail_newinput = str(getContent(URL))



In [32]:
blackmail_newinput = input("Please enter a potentially toxic comment： ")

Please enter a potentially toxic comment： I fuck with your mom


In [33]:
# 將前處理好的input塞進model裡面，predict出原始的prediction
def new_bert_encoder(blackmails_new):
    X_test_tokens = tokenizer(blackmails_new, truncation=True, max_length=200, pad_to_max_length=True)
    return X_test_tokens['input_ids'], X_test_tokens['token_type_ids'], X_test_tokens['attention_mask']
hi = new_bert_encoder(blackmail_newinput)
bert_newinput = np.array([hi]).squeeze()
prediction_raw = model.predict(bert_newinput)
print(prediction_raw)

1/1 [==============================] - 0s 55ms/step
TFSequenceClassifierOutput(loss=None, logits=array([[ 6.4465137],
       [-4.747425 ],
       [-4.8422737]], dtype=float32), hidden_states=None, attentions=None)


In [34]:
# 將原始的醜ouptput轉換成soft_max看得懂的形式，並秀出答案。
a = str(prediction_raw)
b = []
for i in a:
  if i in "0123456789.-]":
    b.append(i)

find_num = [0]
num_list = []
for i in range(3):
  find_num.append(int(b.index("]",find_num[-1]+1)))
  del b[find_num[-1]]
  numbers = str("".join(b[find_num[-2]:find_num[-1]]))
  num_list.append(numbers)

tf_prediction = tensorflow.nn.softmax([[float(num_list[0])],
       [float(num_list[1])],
       [float(num_list[2])]],axis=0).numpy()
print(tf_prediction)
print("The probability that this sentence is a toxic comment is：" + str(tf_prediction[0][0]*100) + "%")
print("sentence: ", blackmail_newinput)

[[9.9997377e-01]
 [1.3756963e-05]
 [1.2512104e-05]]
The probability that this sentence is a toxic comment is：99.99737739562988%
sentence:  I fuck with your mom


In [35]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  769       
                                                                 
Total params: 109,483,009
Trainable params: 109,483,009
Non-trainable params: 0
_________________________________________________________________


In [37]:
from keras.models import load_model
model.save_weights("/content/drive/MyDrive/model_weights.h5")

In [43]:
from transformers import BertTokenizerFast, TFBertForSequenceClassification
import tensorflow as tf

PRETRAINED_MODEL = 'bert-base-uncased'
new_tokenizer = BertTokenizerFast.from_pretrained(PRETRAINED_MODEL)
new_model = TFBertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=1)
new_model.load_weights("/content/drive/MyDrive/model_weights.h5")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
blackmail_newinput = input("Please enter a potentially toxic comment： ")

Please enter a potentially toxic comment： I fuck with your mom


In [47]:
# 將前處理好的input塞進model裡面，predict出原始的prediction
def new_bert_encoder(blackmails_new):
    X_test_tokens = tokenizer(blackmails_new, truncation=True, max_length=200, pad_to_max_length=True)
    return X_test_tokens['input_ids'], X_test_tokens['token_type_ids'], X_test_tokens['attention_mask']
hi = new_bert_encoder(blackmail_newinput)
bert_newinput = np.array([hi]).squeeze()
prediction_raw = new_model.predict(bert_newinput)
print(prediction_raw)

1/1 [==============================] - 3s 3s/step
TFSequenceClassifierOutput(loss=None, logits=array([[ 4.5020337],
       [-2.8723392],
       [-3.3208034]], dtype=float32), hidden_states=None, attentions=None)


In [48]:
a = str(prediction_raw)
b = []
for i in a:
  if i in "0123456789.-]":
    b.append(i)

find_num = [0]
num_list = []
for i in range(3):
  find_num.append(int(b.index("]",find_num[-1]+1)))
  del b[find_num[-1]]
  numbers = str("".join(b[find_num[-2]:find_num[-1]]))
  num_list.append(numbers)

tf_prediction = tensorflow.nn.softmax([[float(num_list[0])],
       [float(num_list[1])],
       [float(num_list[2])]],axis=0).numpy()
print(tf_prediction)
print("The probability that this sentence is a toxic comment is：" + str(tf_prediction[0][0]*100) + "%")
print("sentence: ", blackmail_newinput)

[[9.9897337e-01]
 [6.2647596e-04]
 [4.0007278e-04]]
The probability that this sentence is a toxic comment is：99.89733695983887%
sentence:  I fuck with your mom
